In [1]:
import pickle
import dill
import lzma
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
from jitr.optical_potentials import kduq

In [3]:
import rxmc

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


## read in KDUQ parameter samples

In [4]:
neutron = (1, 0)
proton = (1, 1)

In [5]:
kduq_params_nn = kduq.get_samples_federal(neutron)
kduq_params_pp = kduq.get_samples_federal(proton)

In [6]:
nparams_nn = len(kduq_params_nn[0])
nparams_pp = len(kduq_params_pp[0])

## read in observations and models

In [7]:
from pathlib import Path

evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")

output_dir = Path("./kduq_propagation//")
output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

In [9]:
with open(model_dir / "kduq_nn_model.pkl", "rb") as f:
    kduq_nn_model = dill.load(f)

In [10]:
with open(model_dir / "kduq_pp_model.pkl", "rb") as f:
    kduq_pp_model = dill.load(f)

## propagate KDUQ into constraint observables

In [11]:
yth_nn = []
for x in tqdm(nn_elastic_obs):
    yth_nn.append( np.array([kduq_nn_model(x,*p) for p in kduq_params_nn])) )

  0%|                                                                        | 0/65 [00:00<?, ?it/s]/home/beyerk/Projects/jitr/src/jitr/optical_potentials/kduq.py:396: RuntimeWarning: overflow encountered in exp
  d2 = d2_0 + d2_A / (1 + np.exp((A - d2_A3) / d2_A2))
100%|███████████████████████████████████████████████████████████████| 65/65 [00:58<00:00,  1.10it/s]


In [12]:
yth_pp = []
for x in tqdm(pp_elastic_obs):
    yth_pp.append( np.array([kduq_pp_model(x,*p) for p in kduq_params_pp]))

  0%|                                                                       | 0/160 [00:00<?, ?it/s]/home/beyerk/Projects/jitr/src/jitr/optical_potentials/kduq.py:396: RuntimeWarning: overflow encountered in exp
  d2 = d2_0 + d2_A / (1 + np.exp((A - d2_A3) / d2_A2))
100%|█████████████████████████████████████████████████████████████| 160/160 [02:16<00:00,  1.17it/s]


## write KDUQ posterior predictive distributions to disk

In [13]:
with open(output_dir / "nn_kduq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [14]:
with open(output_dir / "pp_kduq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)